In [156]:
import pandas as pd
import itertools

In [157]:
budburst_observations_df = pd.read_csv('budburst_observations_1689952233.csv')
budburst_observations_df["observation_date"] = pd.to_datetime(budburst_observations_df["observation_date"], format="%m/%d/%Y")

/var/folders/ss/hgshyw356p70rkpdn9mz53kr0000gn/T/ipykernel_7626/2413830639.py:1: DtypeWarning: Columns (8,9,20,29,30,32,33,54,56,59,65) have mixed types. Specify dtype option on import or set low_memory=False.
  budburst_observations_df = pd.read_csv('budburst_observations_1689952233.csv')


In [158]:
budburst_observations_df["observation_year"] = budburst_observations_df["observation_date"].dt.year

In [159]:
budburst_observations_df["observation_quarter"] = budburst_observations_df["observation_date"].dt.quarter

In [160]:
# Data is skewed such that some months have far more observations than other months
budburst_observations_df['observation_quarter'].value_counts()

2    44244
4    29147
3    21764
1    14999
Name: observation_quarter, dtype: int64

In [162]:
start_year = 2017
observations_needed = 20

print(f"""
These are the species that you can use just budburst data to study.

Assumptions:
    - (Arbitrarily) {observations_needed} observations per quarter needed 
    - Starting from {start_year}, each quarter must have the requisite number of species observations
    
Does not take into account geographic location
""")

budburst_observations_df_after_cutoff = budburst_observations_df[budburst_observations_df['observation_year'] >= start_year]
data_percentage_kept = budburst_observations_df_after_cutoff['observation_year'].count() / budburst_observations_df['observation_year'].count()

df = budburst_observations_df_after_cutoff.groupby(['common_name', 'observation_year', 'observation_quarter'])['species_id'].size().reset_index(name='counts')
df = df[df['counts'] >= observations_needed]

year = range(start_year, 2023)
quarter = [1, 2, 3, 4]
complete_list_of_month_year_combinations = list(itertools.product(quarter, year))

potential_species = df['common_name'].unique()
qualifying_species = []

def observation_exists(common_name, observation_quarter, observation_year):
    try:
        df.loc[(df['common_name'] == common_name) & (df['observation_quarter'] == observation_quarter) & (df['observation_year'] == observation_year)].iloc[0]
    except IndexError:
        return False
    return True

def all_observations_exist(common_name):
    for observation_quarter, observation_year in complete_list_of_month_year_combinations:
        if not observation_exists(common_name, observation_quarter, observation_year):
            return False
    
    return True

for common_name in potential_species:
    if all_observations_exist(common_name):
        qualifying_species.append(common_name)
        
print(qualifying_species)


These are the species that you can use just budburst data to study.

Assumptions:
    - (Arbitrarily) 20 observations per quarter needed 
    - Starting from 2017, each quarter must have the requisite number of species observations
    
Does not take into account geographic location

['eastern redbud']
